# Phase 2 project:
## Movie Studio Analysis

# Business Understanding 

A company is interested in entering the movie studio industry. This is based on performances of films from studios such as Warner Bros, Sony, and Paramount Pictures.
They have no background/knowledge on the film industry, but are excited to try it out. They would like an analysis on the movies that have been performing well to make a data-driven decision. 

**Key focus points:**
1. Which genre of movies are performing well in the Box Office
2. How much revenue are they making
3. The ratings of the movies and their respective budget 
4. The runtime of the movies that are performing well

The focus points are to guide the company into ensuring the films produced and released are performing well (ratings) at the Box Office, are a good investment (ROI) and they work well with the given budget. 

**Datasets used:**
- IMDB dataset
- bom.movie_gross dataset
- 


**Outcome:**
- 3 recommendations for the potential new movie studio. This helps guide the company to draw an appropriate and data-driven decision. 


# Data Understanding 

In this section, we'll be going through the datasets and choosing the most appropriate for this analysis/project.

In [1]:
#importing the necessary Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3

The provided datasets are:
- bom.movie_gross
- imdb
- rt.movie_info
- rt.reviews
- tmdb.movies
- tn.movies_budgets

In [2]:
movie_gross = pd.read_csv("C:/Users/PC/Desktop/School work/Projects/Phase 2/Phase-2-Movie-Project/data/bom.movie_gross.csv.gz")
movie_gross.head(2)

,title,studio,domestic_gross,foreign_gross,year
0,Toy Story 3,BV,415000000.0,652000000,2010
1,Alice in Wonderland (2010),BV,334200000.0,691300000,2010


In [3]:
rotten_tomatoes = pd.read_csv("C:/Users/PC/Desktop/School work/Projects/Phase 2/Phase-2-Movie-Project/data/rt.movie_info.tsv.gz", sep = '\t')
rotten_tomatoes.head(2)

,id,synopsis,rating,genre,director,writer,theater_date,dvd_date,currency,box_office,runtime,studio
0,1,"This gritty, fast-paced, and innovative police...",R,Action and Adventure|Classics|Drama,William Friedkin,Ernest Tidyman,"Oct 9, 1971","Sep 25, 2001",NaN,NaN,104 minutes,NaN
1,3,"New York City, not-too-distant-future: Eric Pa...",R,Drama|Science Fiction and Fantasy,David Cronenberg,David Cronenberg|Don DeLillo,"Aug 17, 2012","Jan 1, 2013",$,"600,000",108 minutes,Entertainment One


In [4]:
reviews = pd.read_csv("C:/Users/PC/Desktop/School work/Projects/Phase 2/Phase-2-Movie-Project/data/rt.reviews.tsv.gz", sep = '\t', encoding='cp1252')
reviews.head(2)

,id,review,rating,fresh,critic,top_critic,publisher,date
0,3,A distinctly gallows take on contemporary fina...,3/5,fresh,PJ Nabarro,0,Patrick Nabarro,"November 10, 2018"
1,3,It's an allegory in search of a meaning that n...,NaN,rotten,Annalee Newitz,0,io9.com,"May 23, 2018"


In [5]:
tmdb = pd.read_csv("C:/Users/PC/Desktop/School work/Projects/Phase 2/Phase-2-Movie-Project/data/tmdb.movies.csv.gz")
tmdb.head(2)

,Unnamed: 0,genre_ids,id,original_language,original_title,popularity,release_date,title,vote_average,vote_count
0,0,"[12, 14, 10751]",12444,en,Harry Potter and the Deathly Hallows: Part 1,33.533,2010-11-19,Harry Potter and the Deathly Hallows: Part 1,7.7,10788
1,1,"[14, 12, 16, 10751]",10191,en,How to Train Your Dragon,28.734,2010-03-26,How to Train Your Dragon,7.7,7610


In [6]:
movie_budgets = pd.read_csv("C:/Users/PC/Desktop/School work/Projects/Phase 2/Phase-2-Movie-Project/data/tn.movie_budgets.csv.gz")
movie_budgets.head(2)

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"


In [7]:
imdb = 'C:/Users/PC/Desktop/School work/Projects/Phase 2/Phase-2-Movie-Project/data/im.db'

# Opening up a connection
conn = sqlite3.connect(imdb)

In [8]:
table_name = """
SELECT name 
AS 'Table Names' 
FROM sqlite_master 
WHERE type='table';"""

pd.read_sql(table_name, conn)

,Table Names
0,movie_basics
1,directors
2,known_for
3,movie_akas
4,movie_ratings
5,persons
6,principals
7,writers


In [9]:
query_one= """
SELECT *
FROM movie_basics
LIMIT 2;
"""
pd.read_sql(query_one, conn)

,movie_id,primary_title,original_title,start_year,runtime_minutes,genres
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama"
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama"


In [10]:
query_two= """
SELECT *
FROM movie_ratings
LIMIT 2;
"""
pd.read_sql(query_two, conn)

,movie_id,averagerating,numvotes
0,tt10356526,8.3,31
1,tt10384606,8.9,559


In [11]:
conn.close()

#### From viewing the data above and getting a glimpse of their columns and data in the dataset. The choices are the compulsory IMDb SQLite dataset, bom.movie_gross, and the optional tn. movie_budgets datasets.

# Data Cleaning 

### Box Office Mojo Dataset

In [12]:
movie_gross.head(2)

,title,studio,domestic_gross,foreign_gross,year
0,Toy Story 3,BV,415000000.0,652000000,2010
1,Alice in Wonderland (2010),BV,334200000.0,691300000,2010


In [13]:
# from looking at the first 2 rows of the dataset. We need to remove the year eg 2010 in Alce in Wonderland. And then for ease in merging,
# have all the letters of the title in small/lowercase 

movie_gross['title_new'] = movie_gross['title'].str.lower().str.strip()
movie_gross.head(2)

,title,studio,domestic_gross,foreign_gross,year,title_new
0,Toy Story 3,BV,415000000.0,652000000,2010,toy story 3
1,Alice in Wonderland (2010),BV,334200000.0,691300000,2010,alice in wonderland (2010)


In [14]:
movie_gross.columns

Index(['title', 'studio', 'domestic_gross', 'foreign_gross', 'year',
       'title_new'],
      dtype='object')

In [15]:
# we drop the 'title' column as its not needed

movie_gross = movie_gross.drop('title', axis=1) 
movie_gross.head()

,studio,domestic_gross,foreign_gross,year,title_new
0,BV,415000000.0,652000000,2010,toy story 3
1,BV,334200000.0,691300000,2010,alice in wonderland (2010)
2,WB,296000000.0,664300000,2010,harry potter and the deathly hallows part 1
3,WB,292600000.0,535700000,2010,inception
4,P/DW,238700000.0,513900000,2010,shrek forever after


In [16]:
movie_gross.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3387 entries, 0 to 3386
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   studio          3382 non-null   object 
 1   domestic_gross  3359 non-null   float64
 2   foreign_gross   2037 non-null   object 
 3   year            3387 non-null   int64  
 4   title_new       3387 non-null   object 
dtypes: float64(1), int64(1), object(3)
memory usage: 132.4+ KB


In [17]:
# Convert the foreign gross column to numeric 

movie_gross['foreign_gross'] = pd.to_numeric(movie_gross['foreign_gross'], errors ='coerce')
movie_gross.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3387 entries, 0 to 3386
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   studio          3382 non-null   object 
 1   domestic_gross  3359 non-null   float64
 2   foreign_gross   2032 non-null   float64
 3   year            3387 non-null   int64  
 4   title_new       3387 non-null   object 
dtypes: float64(2), int64(1), object(2)
memory usage: 132.4+ KB


In [18]:
movie_gross.duplicated().value_counts()

False    3387
Name: count, dtype: int64

In [19]:
movie_gross.shape

(3387, 5)

In [20]:
# checking for columns with missing values 
movie_gross.isna().sum().sort_values(ascending = True)

year                 0
title_new            0
studio               5
domestic_gross      28
foreign_gross     1355
dtype: int64

In [21]:
# dropping the few rows in the domestic gross and studio columns

movie_gross = movie_gross.dropna(subset = ['studio', 'domestic_gross'])

# filling the foreign gross column with 0. This is because through research, the column is needed to calculate the worldwide gross
# cannot assume it to be mean due to false numbers. Foreign gross is due to the movies not being launched in other countries, or not being calculated/
# tabulated clealry or lost due to currency difference. 

movie_gross['foreign_gross'] = movie_gross['foreign_gross'].fillna(0)

movie_gross.isna().sum().sort_values(ascending = True)

studio            0
domestic_gross    0
foreign_gross     0
year              0
title_new         0
dtype: int64

In [22]:
# creating and calculating a new column, worldwide gross which is to be similar column to the Numbers dataset

movie_gross['worldwide_gross'] = movie_gross['domestic_gross'] + movie_gross['foreign_gross']
movie_gross.head(2)

,studio,domestic_gross,foreign_gross,year,title_new,worldwide_gross
0,BV,415000000.0,652000000.0,2010,toy story 3,1.067000e+09
1,BV,334200000.0,691300000.0,2010,alice in wonderland (2010),1.025500e+09


### The Numbers Dataset

In [23]:
movie_budgets.head(2)

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"


In [24]:
movie_budgets.columns

Index(['id', 'release_date', 'movie', 'production_budget', 'domestic_gross',
       'worldwide_gross'],
      dtype='object')

In [25]:
movie_budgets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 5782 non-null   int64 
 1   release_date       5782 non-null   object
 2   movie              5782 non-null   object
 3   production_budget  5782 non-null   object
 4   domestic_gross     5782 non-null   object
 5   worldwide_gross    5782 non-null   object
dtypes: int64(1), object(5)
memory usage: 271.2+ KB


In [26]:
movie_budgets.isna().sum()

id                   0
release_date         0
movie                0
production_budget    0
domestic_gross       0
worldwide_gross      0
dtype: int64

In [27]:
movie_budgets.shape

(5782, 6)

In [28]:
#Checking for duplicates in the numbers dataset 

movie_budgets.duplicated().value_counts()

False    5782
Name: count, dtype: int64

The Numbers dataset is quite clean but we need to extract the year from the release date in order to merge the datasets well. As well as the colums need to be converted. 

In [29]:
# create a list with the columns in it

columns = ['production_budget', 'domestic_gross','worldwide_gross']
columns

['production_budget', 'domestic_gross', 'worldwide_gross']

In [30]:
# remove the dollar and commas in the 'production_budget', 'domestic_gross','worldwide_gross' columns before converting to numerical

for col in columns:
    movie_budgets[col] = movie_budgets[col].str.replace(r'[\$,]', '', regex=True)

movie_budgets.head(2)

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,425000000,760507625,2776345279
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,410600000,241063875,1045663875


In [31]:
# converting the 'production_budget', 'domestic_gross','worldwide_gross' columns into numerical values 

for col in columns:
    movie_budgets[col] = pd.to_numeric(movie_budgets[col])

movie_budgets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 5782 non-null   int64 
 1   release_date       5782 non-null   object
 2   movie              5782 non-null   object
 3   production_budget  5782 non-null   int64 
 4   domestic_gross     5782 non-null   int64 
 5   worldwide_gross    5782 non-null   int64 
dtypes: int64(4), object(2)
memory usage: 271.2+ KB


In [32]:
# get the year on its own eg 2010 from the release_date column 

# Converting the release_date to datetime
movie_budgets['release_date'] = pd.to_datetime(movie_budgets['release_date'])

# Creating a new column called year and extracting the year from the column of release_date
movie_budgets['year'] = movie_budgets['release_date'].dt.year
movie_budgets.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross,year
0,1,2009-12-18,Avatar,425000000,760507625,2776345279,2009
1,2,2011-05-20,Pirates of the Caribbean: On Stranger Tides,410600000,241063875,1045663875,2011
2,3,2019-06-07,Dark Phoenix,350000000,42762350,149762350,2019
3,4,2015-05-01,Avengers: Age of Ultron,330600000,459005868,1403013963,2015
4,5,2017-12-15,Star Wars Ep. VIII: The Last Jedi,317000000,620181382,1316721747,2017


In [33]:
# converting the movie title to lowercase letters to match the other datasets

movie_budgets['title_new'] = movie_budgets['movie'].str.lower().str.strip()

# Dropping the extra movie column 

movie_budgets = movie_budgets.drop('movie', axis = 1) 
movie_budgets.head(2)

,id,release_date,production_budget,domestic_gross,worldwide_gross,year,title_new
0,1,2009-12-18,425000000,760507625,2776345279,2009,avatar
1,2,2011-05-20,410600000,241063875,1045663875,2011,pirates of the caribbean: on stranger tides


In [34]:
# dropping the release date column as we have extracted year 

movie_budgets = movie_budgets.drop('release_date', axis = 1)
movie_budgets.head(2)

,id,production_budget,domestic_gross,worldwide_gross,year,title_new
0,1,425000000,760507625,2776345279,2009,avatar
1,2,410600000,241063875,1045663875,2011,pirates of the caribbean: on stranger tides


### IMDB Dataset

In [35]:
imdb = 'C:/Users/PC/Desktop/School work/Projects/Phase 2/Phase-2-Movie-Project/data/im.db'

# Opening up a connection
conn = sqlite3.connect(imdb)

In [36]:
# when looking at the datasets to use, the 2 tables that suit best are movie_rating and movie_basics

query_one= """
SELECT *
FROM movie_basics;
"""
pd.read_sql(query_one, conn)

,movie_id,primary_title,original_title,start_year,runtime_minutes,genres
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama"
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama"
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama"
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy"
...,...,...,...,...,...,...
146139,tt9916538,Kuambil Lagi Hatiku,Kuambil Lagi Hatiku,2019,123.0,Drama
146140,tt9916622,Rodolpho Teóphilo - O Legado de um Pioneiro,Rodolpho Teóphilo - O Legado de um Pioneiro,2015,NaN,Documentary
146141,tt9916706,Dankyavar Danka,Dankyavar Danka,2013,NaN,Comedy
146142,tt9916730,6 Gunn,6 Gunn,2017,116.0,None


In [37]:
query_two= """
SELECT *
FROM movie_ratings;
"""
pd.read_sql(query_two, conn)

,movie_id,averagerating,numvotes
0,tt10356526,8.3,31
1,tt10384606,8.9,559
2,tt1042974,6.4,20
3,tt1043726,4.2,50352
4,tt1060240,6.5,21
...,...,...,...
73851,tt9805820,8.1,25
73852,tt9844256,7.5,24
73853,tt9851050,4.7,14
73854,tt9886934,7.0,5


In [38]:
# joining the 2 tables using the movie_id column 

imdb_q = """
SELECT *
FROM movie_basics 
JOIN "movie_ratings"
ON movie_basics.movie_id = "movie_ratings".movie_id;
"""
imdb = pd.read_sql(imdb_q, conn)

In [39]:
conn.close()

In [40]:
imdb.head(2)

,movie_id,primary_title,original_title,start_year,runtime_minutes,genres,movie_id,averagerating,numvotes
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama",tt0063540,7.0,77
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama",tt0066787,7.2,43


In [41]:
imdb.columns

Index(['movie_id', 'primary_title', 'original_title', 'start_year',
       'runtime_minutes', 'genres', 'movie_id', 'averagerating', 'numvotes'],
      dtype='object')

In [42]:
# In order to drop the duplicate column movie_id, we rename the columns first.

imdb.columns = ['movie_id', 'primary_title', 'original_title', 'start_year',
       'runtime_minutes', 'genres', 'movie_id_x', 'averagerating', 'numvotes']
imdb.columns

Index(['movie_id', 'primary_title', 'original_title', 'start_year',
       'runtime_minutes', 'genres', 'movie_id_x', 'averagerating', 'numvotes'],
      dtype='object')

In [43]:
# drop the unrequired columns; 2nd movie_id, original_title

imdb = imdb.drop(['movie_id_x', 'original_title'], axis = 1)
imdb.head(2)

,movie_id,primary_title,start_year,runtime_minutes,genres,averagerating,numvotes
0,tt0063540,Sunghursh,2013,175.0,"Action,Crime,Drama",7.0,77
1,tt0066787,One Day Before the Rainy Season,2019,114.0,"Biography,Drama",7.2,43


In [44]:
imdb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73856 entries, 0 to 73855
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   movie_id         73856 non-null  object 
 1   primary_title    73856 non-null  object 
 2   start_year       73856 non-null  int64  
 3   runtime_minutes  66236 non-null  float64
 4   genres           73052 non-null  object 
 5   averagerating    73856 non-null  float64
 6   numvotes         73856 non-null  int64  
dtypes: float64(2), int64(2), object(3)
memory usage: 3.9+ MB


In [45]:
imdb.duplicated().value_counts()

False    73856
Name: count, dtype: int64

In [46]:
imdb.shape

(73856, 7)

In [47]:
imdb.isna().sum().sort_values(ascending = True)

movie_id              0
primary_title         0
start_year            0
averagerating         0
numvotes              0
genres              804
runtime_minutes    7620
dtype: int64

In [48]:
# The runtime_minutes column has too many missing values therefore needs to be dropped 
# Fill the missing genre with unknown because it's an object and also cannot make an assumption

# Dropping the column
imdb = imdb.drop(['runtime_minutes'], axis = 1)

# Filling the column with unknown
imdb['genres'] = imdb['genres'].fillna('Unknown')

imdb.isna().sum().sort_values(ascending = True)

movie_id         0
primary_title    0
start_year       0
genres           0
averagerating    0
numvotes         0
dtype: int64

In [49]:
# The start_year needs to be renamed to year and the primary_title to title_new to match the other datasets

# First change the primary title column to lowercase and drop the column as well
imdb['title_new'] = imdb['primary_title'].str.lower().str.strip()
# dropping
imdb = imdb.drop(['primary_title'], axis = 1)

imdb.head(2)

,movie_id,start_year,genres,averagerating,numvotes,title_new
0,tt0063540,2013,"Action,Crime,Drama",7.0,77,sunghursh
1,tt0066787,2019,"Biography,Drama",7.2,43,one day before the rainy season


In [50]:
# rename start_year to year 

imdb = imdb.rename(columns={'start_year': 'year'})
imdb.head(2)

,movie_id,year,genres,averagerating,numvotes,title_new
0,tt0063540,2013,"Action,Crime,Drama",7.0,77,sunghursh
1,tt0066787,2019,"Biography,Drama",7.2,43,one day before the rainy season
